In [1]:
import os
import glob
import numpy as np
import random
from tqdm import tqdm

# =========================================================
# 0. 設定エリア (環境に合わせて調整してください)
# =========================================================
# ファイルパス設定
BASE_DIR = r"C:\Users\fujiw\OneDrive\デスクトップ\ECG_ResNet"
TARGET_DIR = os.path.join(BASE_DIR, "stage2")  # 対象のnpyファイルがあるフォルダ

# データ拡張のハイパーパラメータ
N_MASKS_RANGE = (1, 2)        # 1区間・1波形あたりの欠損箇所数 (最小, 最大)
MASK_PCT_RANGE = (0.02, 0.08) # 欠損させる長さの割合 (2%〜8%)
NOISE_STD = 0.01              # 0付近にする際のノイズの標準偏差

# 保存時のファイル名につける接尾辞
# 例: "12345-0012.npy" -> "12345-0012_aug.npy"
# これにより学習コードの "split('-')[0]" ロジックを壊さずに済みます。
SUFFIX = "_aug"

# =========================================================
# 1. データ拡張関数 (提供されたロジック)
# =========================================================
def augment_waveforms(data, n_masks_range=(1, 2), mask_pct_range=(0.02, 0.08), noise_std=0.01):
    """
    波形データに対して、ランダムな区間を0付近の値に置換（欠損シミュレーション）する関数。
    各特徴量(Row 0-3)とその確信度(Row 4-7)のペアごとに独立して処理を行います。
    """
    augmented = data.copy()
    rows, cols = data.shape
    
    # 1. セグメントの境界を特定 (Row 9 の値が変化する場所)
    row9 = data[9]
    change_indices = np.where(np.diff(row9) != 0)[0] + 1
    segment_boundaries = np.concatenate(([0], change_indices, [cols]))
    
    # 2. 各セグメントに対して処理
    for i in range(len(segment_boundaries) - 1):
        start = segment_boundaries[i]
        end = segment_boundaries[i+1]
        seg_len = end - start
        
        # 極端に短いセグメントはスキップ
        if seg_len < 50:
            continue
            
        # 3. 4つの特徴量・確信度ペア (0/4, 1/5, 2/6, 3/7) それぞれに独立して適用
        for feat_idx in range(4):
            conf_idx = feat_idx + 4
            
            # この波形にいくつの欠損を作るか決定
            n_masks = random.randint(n_masks_range[0], n_masks_range[1])
            
            for _ in range(n_masks):
                # 欠損させる長さを決定 (セグメント長に対する割合)
                pct = random.uniform(mask_pct_range[0], mask_pct_range[1])
                mask_len = int(seg_len * pct)
                mask_len = max(1, mask_len) # 最低1フレーム
                
                # 欠損させる開始位置をランダムに決定
                if seg_len > mask_len:
                    rel_start = random.randint(0, seg_len - mask_len)
                    mask_start = start + rel_start
                    mask_end = mask_start + mask_len
                    
                    # ノイズの生成 (0付近)
                    noise_feat = np.random.normal(0, noise_std, mask_len)
                    noise_conf = np.random.normal(0, noise_std, mask_len)
                    
                    # データに適用
                    augmented[feat_idx, mask_start:mask_end] = noise_feat
                    augmented[conf_idx, mask_start:mask_end] = noise_conf
                    
    return augmented

# =========================================================
# 2. メイン実行処理
# =========================================================
def main():
    if not os.path.exists(TARGET_DIR):
        print(f"Error: Directory not found: {TARGET_DIR}")
        return

    # 全ての .npy ファイルを取得
    all_files = glob.glob(os.path.join(TARGET_DIR, "*.npy"))
    
    # すでに生成済みのファイル（_aug がついているもの）は除外して、二重生成を防ぐ
    target_files = [f for f in all_files if SUFFIX not in f]
    
    print(f"Target Directory: {TARGET_DIR}")
    print(f"Total .npy files found: {len(all_files)}")
    print(f"Files to augment (excluding '{SUFFIX}'): {len(target_files)}")
    print("-" * 50)

    count = 0
    
    # tqdmで進捗を表示しながら実行
    for fpath in tqdm(target_files, desc="Augmenting"):
        try:
            # データの読み込み
            data = np.load(fpath)
            
            # データ拡張の適用
            aug_data = augment_waveforms(
                data, 
                n_masks_range=N_MASKS_RANGE, 
                mask_pct_range=MASK_PCT_RANGE, 
                noise_std=NOISE_STD
            )
            
            # 保存ファイル名の作成
            # 例: C:/.../12345-0012.npy -> C:/.../12345-0012_aug.npy
            fname = os.path.basename(fpath)
            fname_no_ext = os.path.splitext(fname)[0]
            save_name = f"{fname_no_ext}{SUFFIX}.npy"
            save_path = os.path.join(TARGET_DIR, save_name)
            
            # 保存
            np.save(save_path, aug_data)
            count += 1
            
        except Exception as e:
            print(f"\n[Error] Failed to process {fpath}: {e}")

    print("-" * 50)
    print(f"Processing Complete!")
    print(f"Successfully generated {count} new files.")
    print(f"Files are saved in: {TARGET_DIR}")

if __name__ == "__main__":
    # シード値を固定して再現性を確保（必要に応じて変更可）
    random.seed(42)
    np.random.seed(42)
    
    main()

Target Directory: C:\Users\fujiw\OneDrive\デスクトップ\ECG_ResNet\stage2
Total .npy files found: 8793
Files to augment (excluding '_aug'): 8793
--------------------------------------------------


Augmenting: 100%|█████████████| 8793/8793 [00:17<00:00, 506.80it/s]

--------------------------------------------------
Processing Complete!
Successfully generated 8793 new files.
Files are saved in: C:\Users\fujiw\OneDrive\デスクトップ\ECG_ResNet\stage2
